In [1]:
import pandas as pd
from datetime import datetime
from espn_api.football import League

In [2]:
LEAGUE_ID = 48899928
YEAR = datetime.today().year
league = League(league_id=LEAGUE_ID, year=YEAR)

# Weekly Matchup

In [3]:
box_scores = league.box_scores(week=league.currentMatchupPeriod)
box_score = box_scores[0]

In [4]:
def get_team_lineup(lineup_list: list) -> list:
    """
    Gathers certain fields from either
    `BoxScore.home_lineup` or `BoxScore.away_lineup`
    and returns a single dictionary.

    Arguments:
        lineup_list (list): List of `espn_api.football.box_player.BoxPlayer`.

    Returns:
        list: List of dictionaries of players and their attributes.

    """

    team = []
    for player in get_team_lineup:
        data = {}
        data["injuryStatus"] = (
            "-" if player.injuryStatus == "ACTIVE" else player.injuryStatus
        )
        data["name"] = player.name
        data["playerId"] = player.playerId
        data["points"] = player.points
        data["projected_points"] = player.projected_points
        data["position"] = (
            "FLEX" if player.slot_position == "RB/WR/TE" else player.slot_position
        )
        team.append(data)
    return team

In [5]:
matchups = []
for matchup in box_scores:
    data = {}

    away_team = get_team_lineup(box_score.away_lineup)
    data["away_team"] = away_team
    data["away_score"] = box_score.away_score
    data["away_projected"] = box_score.away_projected

    home_team = get_team_lineup(box_score.home_lineup)
    data["home_team"] = home_team
    data["home_score"] = box_score.home_score
    data["home_projected"] = box_score.home_projected

    data["isPlayoffGame"] = str(box_score.matchup_type).lower() == "none"

    team, score = league.least_scored_week()
    data["lowest_scoring_team"] = team.team_name
    data["lowest_scoring_team_score"] = score

    team, score = league.top_scored_week()
    data["highest_scoring_team"] = team.team_name
    data["highest_scoring_team_score"] = score

    matchups.append(data)

# Top Free Agents

In [30]:
available_free_agents = league.free_agents()

In [31]:
free_agents = []
for player in available_free_agents:
    free_agent = {}
    free_agent["name"] = player.name
    free_agent["points"] = player.points
    free_agent["projectedPoints"] = player.projected_points
    free_agent["injuryStatus"] = (
        "-"
        if player.injuryStatus == "ACTIVE" or player.injuryStatus == []
        else player.injuryStatus
    )
    free_agent["position"] = player.position
    free_agent["team"] = player.proTeam
    free_agent["opponent"] = player.pro_opponent
    free_agents.append(free_agent)

free_agents = pd.DataFrame(free_agents)
top_projected_free_agents = (
    free_agents.groupby(["position"])
    .apply(lambda x: x.sort_values(["projectedPoints"], ascending=False))
    .reset_index(drop=True)
)
top_five_projected_free_agents = top_projected_free_agents.groupby("position").head(5)

top_scoring_free_agents = (
    free_agents.groupby(["position"])
    .apply(lambda x: x.sort_values(["points"], ascending=False))
    .reset_index(drop=True)
)
top_five_scoring_free_agents = top_scoring_free_agents.groupby("position").head(5)
top_five_projected_free_agents = top_five_projected_free_agents.to_dict(
    orient="records"
)
top_five_scoring_free_agents = top_five_scoring_free_agents.to_dict(orient="records")

# Standings

In [26]:
standings = []
for idx, team in enumerate(league.standings()):
    standing = {}
    standing["standing"] = team.standing
    standing["team_name"] = team.team_name
    standing["wins"] = team.wins
    standing["losses"] = team.losses
    standing["ties"] = team.ties
    standing["logo_url"] = team.logo_url
    standing["playoff_pct"] = team.playoff_pct
    standing["points_for"] = team.points_for
    standing["points_against"] = team.points_against
    standings.append(standing)

standings = pd.DataFrame(standings)
standings["standing"] = standings["standing"].astype(int)
standings.sort_values(by="standing", ascending=True, inplace=True)
standings["standing"] = standings["standing"].astype(str)
standings = standings.to_dict(orient="records")

# Power Rankings

In [27]:
power_rankings = []
for score, team in league.power_rankings():
    power_ranking = {}
    power_ranking["power_ranking"] = score
    power_ranking["team_name"] = team.team_name
    power_ranking["wins"] = team.wins
    power_ranking["losses"] = team.losses
    power_ranking["ties"] = team.ties
    power_ranking["logo_url"] = team.logo_url
    power_ranking["playoff_pct"] = team.playoff_pct
    power_ranking["points_for"] = team.points_for
    power_ranking["points_against"] = team.points_against
    power_rankings.append(power_ranking)

power_rankings = pd.DataFrame(power_rankings)
power_rankings["power_ranking"] = power_rankings["power_ranking"].astype(float)
power_rankings.sort_values(by="power_ranking", ascending=False, inplace=True)
power_rankings["power_ranking"] = power_rankings["power_ranking"].astype(str)
power_rankings = power_rankings.to_dict(orient="records")